In [6]:
!pip install kaggle

     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.1 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/82.1 kB ? eta -:--:--
     -------------- ----------------------- 30.7/82.1 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 82.1/82.1 kB 460.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.2 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 78.2/78.2 kB 2.2 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105128 sha256=e38e522dbec8fc1ffd00b2a41f184326ff21a2b892e6c8029f598b168762e30e
  Stored in directory: c:\users\prati\appdata\local\pip\cache\wheels\89\bc\52\0d140fc172783e1e72e3af4dfc13015dc154527fafb95e65cd
Successfully 

**Importing the Dependencies**

In [1]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Data Collection- Kaggle API**

In [2]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [3]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [4]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [7]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
  4%|3         | 1.00M/25.7M [00:01<00:33, 766kB/s]
  8%|7         | 2.00M/25.7M [00:01<00:16, 1.50MB/s]
 12%|#1        | 3.00M/25.7M [00:01<00:10, 2.24MB/s]
 16%|#5        | 4.00M/25.7M [00:01<00:07, 3.13MB/s]
 19%|#9        | 5.00M/25.7M [00:02<00:05, 3.89MB/s]
 23%|##3       | 6.00M/25.7M [00:02<00:04, 4.43MB/s]
 27%|##7       | 7.00M/25.7M [00:02<00:03, 4.93MB/s]
 31%|###1      | 8.00M/25.7M [00:02<00:03, 5.50MB/s]
 35%|###5      | 9.00M/25.7M [00:02<00:02, 5.90MB/s]
 39%|###8      | 10.0M/25.7M [00:03<00:03, 4.69MB/s]
 43%|####2     | 11.0M/25.7M [00:03<00:02, 5.20MB/s]
 47%|####6     | 12.0M/25.7M [00:03<00:02, 5.36MB/s]
 51%|#####     | 13.0M/25.7M [00:03<00:02, 5.71MB/s]
 54%|#####4    | 14.0M/25.7M [00:03<00:02, 6.05MB/s]
 58%|#####8    | 15.0M/25.7M [00:03<00:01, 6.49MB/s]
 62%|######2   | 16.0M/25.7M [00:03<00:01, 6.63MB/s]
 66%|######6   | 17.0M/25.7M [00:04<00:01, 6.55MB/s]
 70%|#######   | 18.0M/25.7M [00:04<00:01, 6.54MB/s]
 7

In [8]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [10]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


**Loading teh Dataset**

In [13]:
data = pd.read_csv("IMDB Dataset.csv")

In [14]:
data.shape

(50000, 2)

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [17]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [18]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\prati\AppData\Local\Temp\ipykernel_24696\2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [21]:
# split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [22]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [23]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [24]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [25]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [26]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [27]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [30]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))


In [31]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Training the Model**

In [ ]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 226s 442ms/step - loss: 0.3969 - accuracy: 0.8206 - val_loss: 0.2932 - val_accuracy: 0.8771
Epoch 2/5
500/500 [==============================] - 203s 405ms/step - loss: 0.2666 - accuracy: 0.8934 - val_loss: 0.3097 - val_accuracy: 0.8696
Epoch 3/5
500/500 [==============================] - 203s 407ms/step - loss: 0.2293 - accuracy: 0.9103 - val_loss: 0.3721 - val_accuracy: 0.8669
Epoch 4/5
500/500 [==============================] - 204s 408ms/step - loss: 0.1927 - accuracy: 0.9261 - val_loss: 0.3322 - val_accuracy: 0.8706
Epoch 5/5
500/500 [==============================] - 200s 400ms/step - loss: 0.1661 - accuracy: 0.9356 - val_loss: 0.3249 - val_accuracy: 0.8751


**Model Evaluation**

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 [==============================] - 21s 64ms/step - loss: 0.3050 - accuracy: 0.8834
Test Loss: 0.304972380399704
Test Accuracy: 0.883400022983551


**Building a Predictive System**

In [ ]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [ ]:
# example usage
new_review = "i had low expectations for the movie but it surprised me."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 66ms/step
The sentiment of the review is: positive


In [ ]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 113ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 74ms/step
The sentiment of the review is: negative
